In [37]:
import numpy as np

environment_rows = 5
environment_cols = 5

# Creating a 3D numpy array to hold current Q-values for each state and action pair: Q(s, a)
q_values = np.zeros((environment_rows, environment_cols, 4))

# numeric action codes: 0 = N, 1 = E, 2  = S, 3 = W
actions = ['N', 'E', 'S','W']

In [49]:
# Defining environment
rewards = np.full((environment_rows, environment_cols), -1)

rewards[0,0] = 10
rewards[1,1] = -10
rewards[2,0] = -10

for row in rewards:
    print(row)

[10 -1 -1 -1 -1]
[ -1 -10  -1  -1  -1]
[-10  -1  -1  -1  -1]
[-1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1]


Note: Q-values represent our current estimate of the sum of all future rewards if we were to take a particular action in a particular state

In [39]:
# determines if specific location is a terminal state
def is_terminal(cur_row_idx, cur_col_idx):
    # if the reward is -1 then it is not a terminal state
    if rewards[cur_row_idx, cur_col_idx] == -1: return False
    else: return True

# Epsilon greedy algorithm that will choose which acion to take next
def get_next_action(cur_row_idx, cur_col_idx, epsilon):
    # if a randomly chosen value between 0 and 1 is less than epsilon, choose most promising value from Q-table for this state [90%]
    if np.random.random() < epsilon:
        return np.argmax(q_values[cur_row_idx, cur_col_idx])
    else: # choose random action for exploration
        return np.random.randint(4)
    
# Get next location based on chosen action
def get_next_location(cur_row_idx, cur_col_idx, action_idx):
    new_row_idx = cur_row_idx
    new_col_idx = cur_col_idx
    if actions[action_idx] == 'N' and cur_row_idx > 0:
        new_row_idx -= 1
    elif actions[action_idx] == 'E' and cur_col_idx < environment_cols - 1:
        new_col_idx += 1
    elif actions[action_idx] == 'S' and cur_row_idx < environment_rows - 1:
        new_row_idx += 1
    elif actions[action_idx] == 'W' and cur_col_idx > 0:
        new_col_idx -= 1
    return new_row_idx, new_col_idx

#Get the shortest path between any location within the space that the agent is allowed to travel and the item packaging location.
def get_shortest_path(start_row_index, start_column_index):
  #return immediately if this is an invalid starting location
  if is_terminal(start_row_index, start_column_index):
    return []
  else: #if this is a 'legal' starting location
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    #continue moving along the path until we reach the goal (i.e., the item packaging location)
    while not is_terminal(current_row_index, current_column_index):
      #get the best action to take
      action_index = get_next_action(current_row_index, current_column_index, 1.)
      #move to the next location on the path, and add the new location to the list
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
    return shortest_path
  
  #define a function that will choose a random, non-terminal starting location
def get_starting_location():
  #get a random row and column index
  current_row_index = np.random.randint(environment_rows)
  current_column_index = np.random.randint(environment_cols)
  #continue choosing random row and column indexes until a non-terminal state is identified
  #(i.e., until the chosen state is a 'white square').
  while is_terminal(current_row_index, current_column_index):
    current_row_index = np.random.randint(environment_rows)
    current_column_index = np.random.randint(environment_cols)
  return current_row_index, current_column_index

In [50]:
#define training parameters
epsilon = 0.9 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.9 #discount factor for future rewards
learning_rate = 0.9 #the rate at which the AI agent should learn

#run through 1000 training episodes
for episode in range(1000):
  #get the starting location for this episode
  row_index, column_index = get_starting_location()

  #continue taking actions (i.e., moving) until we reach a terminal state
  #(i.e., until we reach the item packaging area or crash into an item storage location)
  while not is_terminal(row_index, column_index):
    #choose which action to take (i.e., where to move next)
    action_index = get_next_action(row_index, column_index, epsilon)

    #perform the chosen action, and transition to the next state (i.e., move to the next location)
    old_row_index, old_column_index = row_index, column_index #store the old row and column indexes
    row_index, column_index = get_next_location(row_index, column_index, action_index)
    
    #receive the reward for moving to the new state, and calculate the temporal difference
    reward = rewards[row_index, column_index]
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

    #update the Q-value for the previous state and action pair
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete!')

Training complete!


In [51]:
print(get_shortest_path(4, 4))

[[4, 4], [3, 4], [3, 3], [2, 3], [1, 3], [1, 2], [0, 2], [0, 1], [0, 0]]


In [57]:
# Printing Q-table
print("Q-table:")
print("State    |   N       E       S       W")
print("----------------------------------------")
for row in range(environment_rows):
    for col in range(environment_cols):
        print(f"({row},{col})   | ", end="")
        for action_idx, action in enumerate(actions):
            q_value_str = f"{q_values[row, col, action_idx]:.2f}"  # Convert Q-value to string
            padding = max(0, 7 - len(q_value_str))  # Calculate padding to center-align
            print(" " * (padding // 2) + q_value_str + " " * ((padding + 1) // 2), end="")
        print()  # Move to the next line for the next state


Q-table:
State    |   N       E       S       W
----------------------------------------
(0,0)   |  0.00   0.00   0.00   0.00  
(0,1)   |  8.00   6.20   -2.80  10.00 
(0,2)   |  6.20   4.58   4.58   8.00  
(0,3)   |  4.58   3.12   3.12   6.20  
(0,4)   |  3.12   3.12   1.81   4.58  
(1,0)   |  10.00  -2.80  -1.90  8.00  
(1,1)   |  8.00   4.58   4.58   8.00  
(1,2)   |  6.20   3.12   3.12   -2.80 
(1,3)   |  4.58   1.81   1.81   4.58  
(1,4)   |  3.12   1.81   0.63   3.12  
(2,0)   |  8.00   4.58   4.58   6.20  
(2,1)   |  -2.80  3.12   0.63   -2.80 
(2,2)   |  4.58   1.81   1.81   1.81  
(2,3)   |  3.12   0.63   0.63   3.12  
(2,4)   |  1.81   0.61   -0.43  -2.52 
(3,0)   |  -2.79  0.63   -1.36  -0.39 
(3,1)   |  1.81   1.81   -0.43  -0.43 
(3,2)   |  3.12   0.63   0.63   0.63  
(3,3)   |  1.81   -0.43  -0.43  1.81  
(3,4)   |  0.63   -0.43  -1.39  0.63  
(4,0)   |  -0.43  -0.43  -1.38  -1.38 
(4,1)   |  0.63   -1.79  -0.43  -1.29 
(4,2)   |  1.81   -0.44  0.60   1.81  
(4,3)   |  0.6

In [ ]:
# training parameters
epsilon = 0.9 # percentage of the time we take beset action instead of random action
discount_factor = 0.5 #discount factor for future rewards
learning_rate = 0.9 #rate at which the AI agent learns
